In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

%matplotlib inline


In [ ]:
data = pd.read_csv('genetic_variants.csv', sep=',')
data

In [ ]:
data.columns

In [ ]:
data.loc[6628]

## Variables en el datasheet
### Columnas fijas del formato VCF 
- CHROM : identificador en forma RefSeq del cromosoma
- Pos : posición relativa en el cromosoma
- REF : par de base de referencia
- ALT : par de base del alelo

### Frecuencias del alelo según distintas fuentes
- AF_ESP : Frecuencias del alelo en la población según el repositorio GO-ESP
- AF_EXAC : Frecuencias del alelo en la población según el repositorio ExAC
- AF_TGP : Frecuencias del alelo en la población según el repositorio 1000 genomes project

### Tags de ClinVar
- CLNDISDB : enfermedades relacionadas con el par de base 
- CLNDISDBINCL :
- CLNDN : nombre de la enfermedad relacionado con el par de base
- CLNDNINCL :
- CLNHGVS : representación del alelo en formato HGVS
- CLNSIGINCL : A string that describes the variant's clinical significance.  One of the following values may be assigned: 0 unknown, 1 untested, 2 nonpathogenic, 3 probable-nonpathogenic, 4 probable-pathogenic, 5 pathogenic, 6 drug-response, 7 histocompatibility, 255 other.
- CLNVC : efecto del alelo en el par de base
- CLNVI : proteinas relacionadas según fuentes ajenas a ClinVar

### Otras variables
- MC : consecuencia molecular
- ORIGIN :
- SSR : Variant suspect reason code.  The accepted values for this tag are: 0 unspecified, 1 Paralog, 2 byEST, 4 oldAlign, 8 Para_EST, 16 1KG_failed, 1024 other.
- CLASS : La clase que delimita si la alteración es benigna o maligna.
- Allele : 
- Consequence : Consecuencia de la alteración
- IMPACT : Importancia de la consecuencia en la alteración
- SYMBOL : Referencia al simbolo del gen
- Feature_type :
- Feature :
- BIOTYPE : 
- EXON :
- INTRON :
- cDNA_position :
- Protein_position :
- Amino_acids :
- Codons :
- DISTANCE :
- STRAND :
- BAM_EDIT :
- SIFT :
- Polyphen :
- MOTIF_NAME :
- MOTIF_POS :
- HIGH_INF_POS :
- MOTIF_SCORE_CHANGE :
- LoFtool :
- CADD_PHRED : 
- BLOSUM62 :


Parece que tenemos una variable repetida, la representación del alelo, vamos a comprobar si es igual para todos los casos

In [ ]:
(data["Allele"] == data["ALT"]).all()

In [ ]:
data[(data["Allele"] != data["ALT"])]

In [ ]:
not_equals = data[(data["Allele"] == data["ALT"])]["CLNVC"].unique()
equals = data[(data["Allele"] != data["ALT"])]["CLNVC"].unique()  
total = set(not_equals) | set(equals)
[i for i in total if i not in not_equals or i not in equals]

In [ ]:
not_equals = data[(data["Allele"] == data["ALT"])]["IMPACT"].unique()
equals = data[(data["Allele"] != data["ALT"])]["IMPACT"].unique()  
total = set(not_equals) | set(equals)
[i for i in total if i not in not_equals or i not in equals]

Como no vemos ningún tipo de correlacción entre las diferencias de ALT y Allele y el código de referencia no parece tener ningún tipo de procesamiento singular para las consecuencias podemos asumir un error en el input. También vemos que están afectados 3000 variantes, es decir; menos de un 5% de nuestra colección. Vamos a eliminar estos elementos y también los casos de variantes de más de un par de base. El total va eliminar entorno a un 6% de las variantes.

In [ ]:
filtered_Allele = (data["ALT"].str.len()== 1) & (data["Allele"].str.len()== 1) & (data["REF"].str.len()== 1)
data = data[filtered_Allele]
data

Ya podemos eliminar una de las columnas "Allele" o "ALT"

In [ ]:
data = data.drop(columns =["Allele"])
data

In [ ]:
categories = data.select_dtypes(include = ['object'])
categories

Vemos como tenemos categorias que no son clases como la posición o tienen demasiadas clases como las relativas al tag ClinVar

In [ ]:
categories_to_study = [ "Consequence", "IMPACT", "Feature_type", "BIOTYPE", "BAM_EDIT", "SIFT", "PolyPhen", "MOTIF_NAME", "HIGH_INF_POS"]
categories = categories[categories_to_study]
categories = categories.join(data["CLASS"])
categories

In [ ]:
plt.subplots(len(categories_to_study))

    
def plot_categories(categorie):
    data[[categorie, "CLASS"]].groupby([categorie, "CLASS"]).size().unstack().plot(kind='bar', log=True).get_figure()
    
for categorie in categories_to_study:
    plot = plot_categories(categorie)

Vamos a eliminar las clases que no proveen información como HIGH_INF_POS o MOTIF_NAME para evitar que el modelo no se sobreajuste a variables innecesarias.

In [ ]:
data = data.drop(columns =["Feature_type", "BIOTYPE", "BAM_EDIT", "MOTIF_NAME", "HIGH_INF_POS"])
data 

Vamos a observar las variables con el tag ClinVar

In [ ]:
clinVar_categories = [ "CLNDISDB", "CLNDISDBINCL", "CLNDN", "CLNDNINCL", "CLNVC", "CLNVI", "CLNHGVS", "CLNSIGINCL"]
clinVar_data = data[clinVar_categories].join(data["CLASS"])
clinVar_data.groupby("CLASS").nunique()

Si realizamos una búsqueda vemos como las variables CLNDISDB y CLNDN están intrisecamente relacionadas (MedGen:C1843891 -> Spinocerebellar_ataxia_21); de la misma forma CLNDISDBINCL y CLNDNINCL

In [ ]:
clinVar_data[["CLNDISDBINCL", "CLNDNINCL", "CLNDISDB", "CLNDN"]].dropna()

In [ ]:
data[~clinVar_data["CLNSIGINCL"].isna()]

In [ ]:
number_of_CLDNs = clinVar_data["CLNDN"].str.extractall(r"\|?(?P<clndn>\w*)").drop_duplicates().count()
number_of_CLNDNINCLs = clinVar_data["CLNDNINCL"].str.extractall(r"\|?(?P<clndn>\w*)").drop_duplicates().count()
number_of_CLDNs, number_of_CLNDNINCLs

In [ ]:
clinVar_data["CLNDN"].str.extractall(r"\|?(?P<clndn>\w*)").dropna().groupby(level=[0]).size().max()

In [ ]:
clinVar_data["CLNSIGINCL"].dropna().count(), clinVar_data["CLNDN"].dropna().count(), clinVar_data["CLNDNINCL"].dropna().count()

In [ ]:
data["CLNDN"] = clinVar_data["CLNDN"].str.extractall(r"\|?(?P<clndn>\w*)").dropna().groupby(level=[0])["clndn"].apply(list)

De momento vamos a eliminar estas variables ya que tienen un número elevado de clases para los pocos casos que tienen. Excepto CLDND el resto van a ser eliminadas.

In [ ]:
clinVar_categories.remove("CLNDN")
data = data.drop(columns = clinVar_categories)
data

Vamos a realizar el mismo procedimiento que MC

In [ ]:
mc_transformed = data[["MC", "Consequence"]]
mc_transformed["MC"] = mc_transformed["MC"].str.extractall(r"\w+\|(?P<MC>\w+)").dropna().groupby(level=[0])["MC"].apply(list)
mc_transformed

In [ ]:
mc_transformed["MC"] = mc_transformed["MC"].fillna("").apply(list)
mc_transformed[~mc_transformed.apply(lambda row : row["Consequence"] in row["MC"], axis=1)].head(25)

In [ ]:
mc_transformed[~mc_transformed.apply(lambda row : row["Consequence"] in row["MC"], axis=1)]["Consequence"].unique()

In [ ]:
mc_transformed[mc_transformed.apply(lambda row : "&" in row["Consequence"] and row["Consequence"] not in row["MC"] , axis=1)]

Vamos a observar si la posición puede ser un atributo interesante y podríamos binarizarlo según cromosoma.

In [ ]:
mc_transformed["Consequence"] = mc_transformed["Consequence"].str.split("&")
mc_transformed

In [ ]:
def transform_consequence(row):
    if(type(row["Consequence"]) == list and len(row["Consequence"]) > 1):
        return [x for x in row["Consequence"] if x not in row["MC"]]
    return row["Consequence"]
        
consequences = mc_transformed.apply(transform_consequence, axis=1)
mc_transformed[~consequences.apply(lambda x: x if len(x) > 1 else None ).isna()]

Stop gained and splice_region_variant como una única clase

In [ ]:
mc_transformed["Consequence"] = consequences.apply(lambda x: f'{x[0]}&{x[1]}' if len(x) > 1 else x[0])
mc_transformed

In [ ]:
def transform_mc(row):
    mc_list = [x for x in row["MC"] if x not in row["Consequence"]]
    return mc_list if len(mc_list) > 0 else []
        
mc_transformed["MC"] = mc_transformed.apply(transform_mc, axis=1)

In [ ]:
mc_transformed["MC"].apply(lambda x : x if len(x) > 0 else None).dropna()

In [ ]:
mc_transformed["Consequence"].unique()

Teniendo en cuenta que MC sólo tiene varias clases en sólo una sexta parte de los casos vamos a unir Consequence y MC

In [ ]:
data["Consequence"] = mc_transformed.apply(lambda x : [x["Consequence"]] + x["MC"], axis=1)
data = data.drop(columns = ["MC"])
data

Vamos a la distribución de los cromosomas

In [ ]:
data["CHROM"].value_counts()

In [ ]:
DNA_position = data[data["CHROM"] == "2"][["cDNA_position", "CLASS"]]
DNA_position["cDNA_position"] = pd.to_numeric(DNA_position["cDNA_position"])
DNA_position["bin"] = pd.qcut(DNA_position["cDNA_position"], 20)
DNA_position.groupby(["bin", "CLASS"]).size().unstack().plot(kind='bar')

In [ ]:
DNA_position = data[data["CHROM"] == "17"][["cDNA_position", "CLASS"]]
DNA_position["cDNA_position"] = pd.to_numeric(DNA_position["cDNA_position"])
DNA_position["bin"] = pd.qcut(DNA_position["cDNA_position"], 20)
DNA_position.groupby(["bin", "CLASS"]).size().unstack().plot(kind='bar')

Puede ser un atributo interesante, vamos a realizar la transformación por cada cromosoma. Pero antes vamos a ver cómo se correlacionan estas variables.

El valor puede estar entre dos valores, vamos a tomar el inferior y si está a ? el superior

In [ ]:
def get_value_position(position):
    if type(position) == str and len(position.split("-")) == 2:
        return position.split("-")[0] if position.split("-")[0] != "?" else position.split("-")[1]
    return position
position = data[["cDNA_position", "CDS_position", "Protein_position"]].copy()
position["cDNA_position"] =  position["cDNA_position"].apply(get_value_position)
position["cDNA_position"] =  pd.to_numeric(position["cDNA_position"])
position["CDS_position"] =  position["CDS_position"].apply(get_value_position)
position["CDS_position"] =  pd.to_numeric(position["CDS_position"])
position["Protein_position"] =  position["Protein_position"].apply(get_value_position)
position["Protein_position"] =  pd.to_numeric(position["Protein_position"])

plt.figure(figsize = (3, 3))
sns.heatmap(position.corr(), annot = True, linewidths=.5, cmap = plt.cm.cool)

In [ ]:
position.std()

Eliminamos las variables correlacionadas cDNA_position y Protein_position

In [ ]:
data = data.drop(columns = ["cDNA_position", "Protein_position"])
data

In [ ]:
#data["CDS_position"] =  data["CDS_position"].apply(get_value_position)
#data["CDS_position"] = pd.to_numeric(data["CDS_position"])

#for chrom in data["CHROM"].unique():
#    selected_chrom = data[data["CHROM"] == chrom]
#    print(chrom)
#    pd.qcut(selected_chrom["CDS_position"], 20)


Hay un problema con los cromosomas mitocondrial

Como hay muy pocos casos de ADN mitocondrial, podríamos eliminarlo por el momento.

In [ ]:
data[data["CHROM"] == "MT"]

Normalize positions for each chromosomes to avoid the differences in the location of chromosomes with longer positions.

In [ ]:
data = data[data["CHROM"] != "MT"]

normalized_positions = []
data["CDS_position"] =  data["CDS_position"].apply(get_value_position)
data["CDS_position"] = pd.to_numeric(data["CDS_position"])

for chrom in data["CHROM"].unique():
    selected_chrom = data[data["CHROM"] == chrom]["CDS_position"]
    min_max = (selected_chrom-selected_chrom.min())/(selected_chrom.max()-selected_chrom.min())
    normalized_positions.append(min_max)

indexes = []
normalized = []

for positions_by_chrom in normalized_positions:
    indexes.extend(positions_by_chrom.index)
    normalized.extend(positions_by_chrom.array)

data["CDS_position"] = pd.Series(index=indexes, data=normalized)
data

Vamos a partir la columna de aminoácidos y a eliminar la de codones ya que parte de la información está incluida en los aminoácidos.

In [ ]:
def split_aminoacids(aminoacid_chain):
    aminoacids = aminoacid_chain.split("/")
    if(len(aminoacids) < 2):
        return [aminoacid_chain]
    return aminoacids
amino_acids_pairs = data["Amino_acids"].dropna().apply(split_aminoacids)

In [ ]:
data["Amino_acids_target"] = amino_acids_pairs.transform(lambda x: x[1] if len(x) > 1 else 'None') 
data["Amino_acids"] = amino_acids_pairs.transform(lambda x: x[0] if len(x) > 0 else 'None') 
data

In [ ]:
data = data.drop(columns = "Codons")
data

Vamos a ver si el símbolo es una variable relevante.

In [ ]:
gene_ct = pd.crosstab(data.SYMBOL, data.CLASS, margins=True)
gene_ct.drop('All', axis=0, inplace=True)

gene_ct = gene_ct.sort_values(by='All', ascending=False).head(50)
gene_ct.drop('All', axis=1, inplace=True)

gene_ct.plot.bar(stacked=True, figsize=(16, 8));

Mismo comportamiento con la variable FEATURE

In [ ]:
gene_ct = pd.crosstab(data.Feature, data.CLASS, margins=True)
gene_ct.drop('All', axis=0, inplace=True)

gene_ct = gene_ct.sort_values(by='All', ascending=False).head(50)
gene_ct.drop('All', axis=1, inplace=True)

gene_ct.plot.bar(stacked=True, figsize=(12, 4));

In [ ]:
data.describe(include='all')

Vemos como AF_ESP, AF_EXAC y AF_TGP tienen características muy similares, vamos  convertirlos en un valor ya que es la misma variable pero tomada de distintas fuentes, hemos optado por aplicar la mediana ya que la media puede verse muy afectada si hay una gran diferencia.

In [ ]:
frecuencies_columns = ["AF_ESP", "AF_EXAC", "AF_TGP"]
frecuency = data[frecuencies_columns]
data["AF"] = frecuency.median(axis=1)
data

In [ ]:
data = data.drop(columns = frecuencies_columns)
data

In [ ]:
data.info()

Vemos como tenemos un número importante de variables que en la mayoría de casos son nulos, vamos a eliminar estas variables siempre y cuando no superen un límite.

In [ ]:
percentage_na = data.isnull().sum().apply(lambda x: x/data.shape[0]*100)
percentage_na

In [ ]:
applied_na = percentage_na[percentage_na > 75]
columns_to_delete = applied_na.index.to_numpy()
columns_to_delete

Vamos a quitar el Intron porque posiblemente tenga relación con el EXON

In [ ]:
columns_to_delete = columns_to_delete[columns_to_delete != "INTRON"] 
columns_to_delete

In [ ]:
data = data.drop(columns=columns_to_delete)
data

PolyPhen ya tiene una clase unknown que podemos emplear para el resto de elementos que faltan

In [ ]:
data["PolyPhen"] = data["PolyPhen"].fillna("unknown")
data["PolyPhen"].unique()

Para los intrones y exones vamos a suponer una localización en el intron

In [ ]:
data["EXON"].unique(), data["INTRON"].unique()

In [ ]:
data[data["INTRON"].notna() & data["EXON"].notna()]

In [ ]:
data[data["INTRON"].isna() & data["EXON"].isna()].info()

In [ ]:
incoherent_indexes = data["INTRON"].isna() & data["EXON"].isna()
data = data[~incoherent_indexes]
data

In [ ]:
gene_ct = pd.crosstab(data.EXON, data.CLASS, margins=True)
gene_ct.drop('All', axis=0, inplace=True)

gene_ct = gene_ct.sort_values(by='All', ascending=False).head(50)
gene_ct.drop('All', axis=1, inplace=True)

gene_ct.plot.bar(stacked=True, figsize=(12, 4));

In [ ]:
gene_ct = pd.crosstab(data.INTRON, data.CLASS, margins=True)
gene_ct.drop('All', axis=0, inplace=True)

gene_ct = gene_ct.sort_values(by='All', ascending=False).head(50)
gene_ct.drop('All', axis=1, inplace=True)

gene_ct.plot.bar(stacked=True, figsize=(12, 4));

Vamos a marcar la localización como una fracción y a unir ambas columnas

In [ ]:
def intro_exon_to_float(value):
    if type(value) == str:
        values = value.split("/")
        return float(values[0])/float(values[1])
    return value

data["IS_EXON"] = ~data["EXON"].isna()
data["EXON"] = data["EXON"].transform(intro_exon_to_float)
data["INTRON"] = data["INTRON"].transform(intro_exon_to_float)
data["EXON_INTRON"] = data["EXON"].fillna(0)+data["INTRON"].fillna(0)

data = data.drop(columns = ["EXON", "INTRON"])
data

Vamos a observar las correlaciones en el código

In [ ]:
plt.figure(figsize = (17, 17))
sns.heatmap(data.corr(), annot = True, linewidths=.5, cmap = plt.cm.cool)

Como vemos que CADD_PHRED y CADD_RAW están correlacionadas, vamos a eliminar la que menos varianza tenga

In [ ]:
cadd_data = data[["CADD_PHRED", "CADD_RAW"]]
cadd_data =(cadd_data-cadd_data.min())/(cadd_data.max()-cadd_data.min())
cadd_data.std()

In [ ]:
data = data.drop(columns=["CADD_RAW"])
data

También vemos que hay una fuerte relación entre LoFTool - POS y EXON_INTRON - CDS_position

In [ ]:
corr_by_chrom = data[data["CHROM"] == "2"].corr()
plt.figure(figsize = (17, 17))
sns.heatmap(corr_by_chrom, annot = True, linewidths=.5, cmap = plt.cm.cool)

In [ ]:
sns.scatterplot(data["CDS_position"], data["LoFtool"])

Como no vemos una tendencia clara vamos a proseguir con esta variable pese a que haya una correlación del 36%

In [ ]:
sns.scatterplot(data["CDS_position"], data["EXON_INTRON"])

Tiene sentido que no haya posiciones elevadas en la proteina para los segmentos de exon o intron donde se encuentran más cerca del inicio.

In [71]:
data

,CHROM,POS,REF,ALT,CLNDN,ORIGIN,CLASS,Consequence,IMPACT,SYMBOL,Feature,CDS_position,Amino_acids,STRAND,SIFT,PolyPhen,LoFtool,CADD_PHRED,BLOSUM62,Amino_acids_target,AF,IS_EXON,EXON_INTRON
0,1,1168180,G,C,[not_specified],1,0,[missense_variant],MODERATE,B3GALT6,NM_080605.3,0.033481,E,1.0,tolerated,benign,NaN,1.053,2.0,D,0.10020,True,1.000000
1,1,1470752,G,A,"[Spinocerebellar_ataxia_21, not_provided]",1,0,[missense_variant],MODERATE,TMEM240,NM_001114748.1,0.032646,P,-1.0,deleterious_low_confidence,benign,NaN,31.000,-3.0,L,0.00000,True,1.000000
2,1,1737942,A,G,"[Strabismus, Nystagmus, Hypothyroidism, Intell...",35,1,"[missense_variant, 5_prime_UTR_variant]",MODERATE,GNB1,NM_002074.4,0.015295,I,-1.0,deleterious,probably_damaging,NaN,28.100,-1.0,T,0.00000,True,0.500000
3,1,2160305,G,A,"[Shprintzen, Goldberg_syndrome, not_provided]",33,0,[missense_variant],MODERATE,SKI,XM_005244775.1,0.006362,G,1.0,NaN,unknown,NaN,22.500,NaN,S,0.00000,True,0.142857
4,1,2160305,G,T,"[Shprintzen, Goldberg_syndrome]",33,0,[missense_variant],MODERATE,SKI,XM_005244775.1,0.006362,G,1.0,NaN,unknown,NaN,24.700,-3.0,C,0.00000,True,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65183,X,154158201,T,G,"[Hereditary_factor_VIII_deficiency_disease, no...",1,0,[synonymous_variant],LOW,F8,NM_000132.3,0.354794,S,-1.0,NaN,unknown,0.00158,0.105,NaN,None,0.13923,True,0.538462
65184,X,154159118,C,T,"[not_specified, Hemophilia_A, _FVIII_Deficiency]",1,1,[missense_variant],MODERATE,F8,NM_000132.3,0.270573,V,-1.0,tolerated,benign,0.00158,0.002,3.0,I,0.00130,True,0.538462
65185,X,154194886,C,T,"[not_specified, Hemophilia_A, _FVIII_Deficiency]",1,0,[synonymous_variant],LOW,F8,NM_000132.3,0.099651,A,-1.0,NaN,unknown,0.00158,12.850,NaN,None,0.01110,True,0.307692
65186,X,154490187,T,C,"[Non, syndromic_X, linked_intellectual_disabil...",1,0,[synonymous_variant],LOW,RAB39B,NM_171998.2,0.049780,T,-1.0,NaN,unknown,NaN,0.130,NaN,None,0.00030,True,1.000000


In [72]:
data.to_csv('data_filtered.csv')